# data prep road accident dataset       

In [3]:
import pandas as pd
import numpy as np


from my_modules import my_functions as mybib
from my_modules import project_functions as pr


%load_ext autoreload
%autoreload 2

### Import berlin road accident files and investigate table structure

In [4]:
accidents18 = pd.read_csv('../data/input/road_accidents/berlin_road_accidents_2018.csv', sep=';', encoding = 'latin1')
print('2018')
display(accidents18.head())

accidents19 = pd.read_csv('../data/input/road_accidents/berlin_road_accidents_2019.csv', sep=';', encoding = 'latin1')
print('2019')
display(accidents19.head())

accidents20 = pd.read_csv('../data/input/road_accidents/berlin_road_accidents_2020.csv', sep=';', encoding = 'latin1')
print('2020')
display(accidents20.head())

accidents21 = pd.read_csv('../data/input/road_accidents/berlin_road_accidents_2021.csv', sep=';', encoding = 'latin1')
print('2021')
display(accidents21.head())

2018


,OBJECTID,LAND,BEZ,LOR,STRASSE,LOR_ab_2021,UJAHR,UMONAT,USTUNDE,UWOCHENTAG,...,IstPKW,IstFuss,IstKrad,IstGkfz,IstSonstig,STRZUSTAND,LINREFX,LINREFY,XGCSWGS84,YGCSWGS84
0,112695,11,2,2050602.0,Samariterviertel,2500729.0,2018,1,15,4,...,1,1,0,0,0,1,"803604,1562","5827580,738","13,4750178","52,51359681"
1,112705,11,12,12304314.0,Ziekowstraße/Freie Scholle,12500824.0,2018,1,11,2,...,1,0,0,0,0,0,"790639,5854","5835009,589","13,29102205","52,58725906"
2,112726,11,2,2040501.0,Barnimkiez,2400520.0,2018,1,9,3,...,1,1,0,0,0,0,"799827,9618","5828733,523","13,42057818","52,52601854"
3,112737,11,7,7020202.0,Volkspark (Rudolf-Wilde-Park),7200308.0,2018,1,17,2,...,1,1,0,0,0,0,"795223,509","5823525,402","13,34828776","52,48184447"
4,112747,11,3,3020209.0,Niederschönhausen,3200206.0,2018,1,15,4,...,0,1,0,0,0,1,"798261,3849","5835047,26","13,40322797","52,58347154"


2019


,OBJECTID,LAND,BEZ,LOR,STRASSE,UJAHR,UMONAT,USTUNDE,UWOCHENTAG,UKATEGORIE,...,IstPKW,IstFuss,IstKrad,IstGkfz,IstSonstige,USTRZUSTAND,LINREFX,LINREFY,XGCSWGS84,YGCSWGS84
0,49090,11,12,12301203,Wittenau Süd,2019,1,13,6,3,...,1,0,0,0,0,1,"794062,2837","5835083,823","13,34146","52,58609"
1,49091,11,3,3040818,Pankow Süd,2019,1,9,5,3,...,1,0,0,0,0,0,"799130,4007","5832327,415","13,41356","52,55862"
2,49093,11,12,12103115,Breitkopfbecken,2019,3,21,6,3,...,0,0,0,0,0,0,"795437,613","5833549,454","13,36034","52,57159"
3,49096,11,6,6040703,Nikolassee,2019,1,7,6,2,...,1,1,0,0,0,1,"786714,3754","5817042,137","13,21777","52,42825"
4,49097,11,7,7030303,Grazer Platz,2019,2,15,3,3,...,1,0,0,0,0,0,"796074,3342","5822724,905","13,36007","52,47421"


2020


,OBJECTID,LAND,BEZ,LOR,LOR_ab_2021,UJAHR,UMONAT,USTUNDE,UWOCHENTAG,UKATEGORIE,...,IstPKW,IstFuss,IstKrad,IstGkfz,IstSonstige,USTRZUSTAND,LINREFX,LINREFY,XGCSWGS84,YGCSWGS84
0,3187,11,8,8010510.0,8100521.0,2020,1,11,4,2,...,1,0,0,1,0,1,"802599,5332","5821795,373","13,45500598","52,4623009"
1,3198,11,4,4041137.0,4400727.0,2020,1,2,3,2,...,0,0,0,1,0,1,"791264,2064","5824629,955","13,29111618","52,49387309"
2,3215,11,4,4041239.0,4400830.0,2020,1,7,1,3,...,0,0,0,0,1,1,"792294,8083","5823598,115","13,30534822","52,48407191"
3,3224,11,2,2040502.0,2400521.0,2020,1,22,6,2,...,1,0,0,0,0,0,"801024,2746","5827862,923","13,43737099","52,517556"
4,3241,11,4,4041137.0,4400727.0,2020,1,17,1,3,...,0,0,1,0,0,1,"791889,2861","5824003,006","13,29974796","52,48791866"


2021


,OBJECTID,LAND,BEZ,LOR_ab_2021,UJAHR,UMONAT,USTUNDE,UWOCHENTAG,UKATEGORIE,UART,...,IstPKW,IstFuss,IstKrad,IstGkfz,IstSonstige,USTRZUSTAND,LINREFX,LINREFY,XGCSWGS84,YGCSWGS84
0,219249,11,3,3701658.0,2021,11,18,2,3,0,...,1,0,1,0,0,1,"800202,0742","5829640,204","13,42689483","52,53393955"
1,219248,11,7,7501134.0,2021,12,19,7,3,6,...,1,1,0,0,0,1,"798479,5317","5819049,219","13,39209027","52,43995086"
2,219247,11,4,4100101.0,2021,12,17,4,3,5,...,1,0,0,0,0,0,"793352,6128","5829680,195","13,32624193","52,53802784"
3,219246,11,4,4501041.0,2021,12,15,7,3,5,...,1,0,1,0,0,1,"792950,0395","5825362,081","13,31652064","52,49953417"
4,219243,11,11,11501339.0,2021,12,9,5,3,3,...,1,0,0,0,1,2,"807182,01","5825602,793","13,52575163","52,49386714"


### check columnnames 

In [5]:
accidents18 = mybib.clean_column_names(accidents18)
accidents19 = mybib.clean_column_names(accidents19)
accidents20 = mybib.clean_column_names(accidents20)
accidents21 = mybib.clean_column_names(accidents21)

#TODO: add a loop for printing
print('2018')
display(accidents18.head())
print(accidents18.shape)
print('2019')
display(accidents19.head())
print(accidents19.shape)
print('2020')
display(accidents20.head())
print(accidents20.shape)
print('2021')
display(accidents21.head())
print(accidents21.shape)

total = len(accidents21) + len(accidents20) + len(accidents19) + len(accidents18)
print(total)


2018


,objectid,land,bez,lor,strasse,lor_ab_2021,ujahr,umonat,ustunde,uwochentag,...,istpkw,istfuss,istkrad,istgkfz,istsonstig,strzustand,linrefx,linrefy,xgcswgs84,ygcswgs84
0,112695,11,2,2050602.0,Samariterviertel,2500729.0,2018,1,15,4,...,1,1,0,0,0,1,"803604,1562","5827580,738","13,4750178","52,51359681"
1,112705,11,12,12304314.0,Ziekowstraße/Freie Scholle,12500824.0,2018,1,11,2,...,1,0,0,0,0,0,"790639,5854","5835009,589","13,29102205","52,58725906"
2,112726,11,2,2040501.0,Barnimkiez,2400520.0,2018,1,9,3,...,1,1,0,0,0,0,"799827,9618","5828733,523","13,42057818","52,52601854"
3,112737,11,7,7020202.0,Volkspark (Rudolf-Wilde-Park),7200308.0,2018,1,17,2,...,1,1,0,0,0,0,"795223,509","5823525,402","13,34828776","52,48184447"
4,112747,11,3,3020209.0,Niederschönhausen,3200206.0,2018,1,15,4,...,0,1,0,0,0,1,"798261,3849","5835047,26","13,40322797","52,58347154"


(13652, 25)
2019


,objectid,land,bez,lor,strasse,ujahr,umonat,ustunde,uwochentag,ukategorie,...,istpkw,istfuss,istkrad,istgkfz,istsonstige,ustrzustand,linrefx,linrefy,xgcswgs84,ygcswgs84
0,49090,11,12,12301203,Wittenau Süd,2019,1,13,6,3,...,1,0,0,0,0,1,"794062,2837","5835083,823","13,34146","52,58609"
1,49091,11,3,3040818,Pankow Süd,2019,1,9,5,3,...,1,0,0,0,0,0,"799130,4007","5832327,415","13,41356","52,55862"
2,49093,11,12,12103115,Breitkopfbecken,2019,3,21,6,3,...,0,0,0,0,0,0,"795437,613","5833549,454","13,36034","52,57159"
3,49096,11,6,6040703,Nikolassee,2019,1,7,6,2,...,1,1,0,0,0,1,"786714,3754","5817042,137","13,21777","52,42825"
4,49097,11,7,7030303,Grazer Platz,2019,2,15,3,3,...,1,0,0,0,0,0,"796074,3342","5822724,905","13,36007","52,47421"


(13390, 24)
2020


,objectid,land,bez,lor,lor_ab_2021,ujahr,umonat,ustunde,uwochentag,ukategorie,...,istpkw,istfuss,istkrad,istgkfz,istsonstige,ustrzustand,linrefx,linrefy,xgcswgs84,ygcswgs84
0,3187,11,8,8010510.0,8100521.0,2020,1,11,4,2,...,1,0,0,1,0,1,"802599,5332","5821795,373","13,45500598","52,4623009"
1,3198,11,4,4041137.0,4400727.0,2020,1,2,3,2,...,0,0,0,1,0,1,"791264,2064","5824629,955","13,29111618","52,49387309"
2,3215,11,4,4041239.0,4400830.0,2020,1,7,1,3,...,0,0,0,0,1,1,"792294,8083","5823598,115","13,30534822","52,48407191"
3,3224,11,2,2040502.0,2400521.0,2020,1,22,6,2,...,1,0,0,0,0,0,"801024,2746","5827862,923","13,43737099","52,517556"
4,3241,11,4,4041137.0,4400727.0,2020,1,17,1,3,...,0,0,1,0,0,1,"791889,2861","5824003,006","13,29974796","52,48791866"


(11810, 24)
2021


,objectid,land,bez,lor_ab_2021,ujahr,umonat,ustunde,uwochentag,ukategorie,uart,...,istpkw,istfuss,istkrad,istgkfz,istsonstige,ustrzustand,linrefx,linrefy,xgcswgs84,ygcswgs84
0,219249,11,3,3701658.0,2021,11,18,2,3,0,...,1,0,1,0,0,1,"800202,0742","5829640,204","13,42689483","52,53393955"
1,219248,11,7,7501134.0,2021,12,19,7,3,6,...,1,1,0,0,0,1,"798479,5317","5819049,219","13,39209027","52,43995086"
2,219247,11,4,4100101.0,2021,12,17,4,3,5,...,1,0,0,0,0,0,"793352,6128","5829680,195","13,32624193","52,53802784"
3,219246,11,4,4501041.0,2021,12,15,7,3,5,...,1,0,1,0,0,1,"792950,0395","5825362,081","13,31652064","52,49953417"
4,219243,11,11,11501339.0,2021,12,9,5,3,3,...,1,0,0,0,1,2,"807182,01","5825602,793","13,52575163","52,49386714"


(11267, 23)
50119


### manually change wrong column names and add missing columns 

In [6]:
# add empty lor_ab_2021 column on accidents19
#TODO: cleaning function
accidents18.rename(columns={'strzustand':'ustrzustand'}, inplace=True)
accidents18.rename(columns={'istsonstig':'istsonstige'}, inplace=True)
accidents19.insert(5, 'lor_ab_2021', '')
accidents20.insert(4, 'strasse', '')
accidents21.insert(3, 'lor', '')
accidents21.insert(4, 'strasse', '')


columns = len(accidents18.columns)

for i in range(columns):
    print(i)
    print(accidents18.columns[i])
    print(accidents19.columns[i])
    print(accidents20.columns[i])
    print(accidents21.columns[i])

0
objectid
objectid
objectid
objectid
1
land
land
land
land
2
bez
bez
bez
bez
3
lor
lor
lor
lor
4
strasse
strasse
strasse
strasse
5
lor_ab_2021
lor_ab_2021
lor_ab_2021
lor_ab_2021
6
ujahr
ujahr
ujahr
ujahr
7
umonat
umonat
umonat
umonat
8
ustunde
ustunde
ustunde
ustunde
9
uwochentag
uwochentag
uwochentag
uwochentag
10
ukategorie
ukategorie
ukategorie
ukategorie
11
uart
uart
uart
uart
12
utyp1
utyp1
utyp1
utyp1
13
ulichtverh
ulichtverh
ulichtverh
ulichtverh
14
istrad
istrad
istrad
istrad
15
istpkw
istpkw
istpkw
istpkw
16
istfuss
istfuss
istfuss
istfuss
17
istkrad
istkrad
istkrad
istkrad
18
istgkfz
istgkfz
istgkfz
istgkfz
19
istsonstige
istsonstige
istsonstige
istsonstige
20
ustrzustand
ustrzustand
ustrzustand
ustrzustand
21
linrefx
linrefx
linrefx
linrefx
22
linrefy
linrefy
linrefy
linrefy
23
xgcswgs84
xgcswgs84
xgcswgs84
xgcswgs84
24
ygcswgs84
ygcswgs84
ygcswgs84
ygcswgs84


In [7]:
#check if columnnames are equal

# *dataframes allows to enter multiple dataframes
def column_name_check(*dataframes):
    check = set(dataframes[0].columns)
    
    for df in dataframes[1:]:
        if set(df.columns)!= check:
            return False
    return True 

print(column_name_check(accidents18, accidents19, accidents20, accidents21))

True


### concat the 4 datasets together  

In [8]:
accident_data = pd.concat([accidents18, accidents19, accidents20, accidents21], axis=0)
accident_data.shape

accident_data = accident_data.drop(columns=['land'])
display(accident_data.head())
accident_data.dtypes


,objectid,bez,lor,strasse,lor_ab_2021,ujahr,umonat,ustunde,uwochentag,ukategorie,...,istpkw,istfuss,istkrad,istgkfz,istsonstige,ustrzustand,linrefx,linrefy,xgcswgs84,ygcswgs84
0,112695,2,2050602.0,Samariterviertel,2500729.0,2018,1,15,4,3,...,1,1,0,0,0,1,"803604,1562","5827580,738","13,4750178","52,51359681"
1,112705,12,12304314.0,Ziekowstraße/Freie Scholle,12500824.0,2018,1,11,2,3,...,1,0,0,0,0,0,"790639,5854","5835009,589","13,29102205","52,58725906"
2,112726,2,2040501.0,Barnimkiez,2400520.0,2018,1,9,3,3,...,1,1,0,0,0,0,"799827,9618","5828733,523","13,42057818","52,52601854"
3,112737,7,7020202.0,Volkspark (Rudolf-Wilde-Park),7200308.0,2018,1,17,2,3,...,1,1,0,0,0,0,"795223,509","5823525,402","13,34828776","52,48184447"
4,112747,3,3020209.0,Niederschönhausen,3200206.0,2018,1,15,4,3,...,0,1,0,0,0,1,"798261,3849","5835047,26","13,40322797","52,58347154"


objectid        int64
bez             int64
lor            object
strasse        object
lor_ab_2021    object
ujahr           int64
umonat          int64
ustunde         int64
uwochentag      int64
ukategorie      int64
uart            int64
utyp1           int64
ulichtverh      int64
istrad          int64
istpkw          int64
istfuss         int64
istkrad         int64
istgkfz         int64
istsonstige    object
ustrzustand    object
linrefx        object
linrefy        object
xgcswgs84      object
ygcswgs84      object
dtype: object

In [9]:

accident_data = accident_data.fillna('0')
accident_data = accident_data.replace("",'0')

mybib.nullTable(accident_data)
display(accident_data)


,header_name,percent_nulls
0,objectid,0.0
1,bez,0.0
2,lor,0.0
3,strasse,0.0
4,lor_ab_2021,0.0
5,ujahr,0.0
6,umonat,0.0
7,ustunde,0.0
8,uwochentag,0.0
9,ukategorie,0.0


objectid       0
bez            0
lor            0
strasse        0
lor_ab_2021    0
ujahr          0
umonat         0
ustunde        0
uwochentag     0
ukategorie     0
uart           0
utyp1          0
ulichtverh     0
istrad         0
istpkw         0
istfuss        0
istkrad        0
istgkfz        0
istsonstige    0
ustrzustand    0
linrefx        0
linrefy        0
xgcswgs84      0
ygcswgs84      0
dtype: int64

,objectid,bez,lor,strasse,lor_ab_2021,ujahr,umonat,ustunde,uwochentag,ukategorie,...,istpkw,istfuss,istkrad,istgkfz,istsonstige,ustrzustand,linrefx,linrefy,xgcswgs84,ygcswgs84
0,112695,2,2050602.0,Samariterviertel,2500729.0,2018,1,15,4,3,...,1,1,0,0,0,1,"803604,1562","5827580,738","13,4750178","52,51359681"
1,112705,12,12304314.0,Ziekowstraße/Freie Scholle,12500824.0,2018,1,11,2,3,...,1,0,0,0,0,0,"790639,5854","5835009,589","13,29102205","52,58725906"
2,112726,2,2040501.0,Barnimkiez,2400520.0,2018,1,9,3,3,...,1,1,0,0,0,0,"799827,9618","5828733,523","13,42057818","52,52601854"
3,112737,7,7020202.0,Volkspark (Rudolf-Wilde-Park),7200308.0,2018,1,17,2,3,...,1,1,0,0,0,0,"795223,509","5823525,402","13,34828776","52,48184447"
4,112747,3,3020209.0,Niederschönhausen,3200206.0,2018,1,15,4,3,...,0,1,0,0,0,1,"798261,3849","5835047,26","13,40322797","52,58347154"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11262,112174,4,0,0,4300414.0,2021,3,16,4,3,...,1,1,0,0,0,1,"791505,4105","5826555,664","13,29634478","52,51100814"
11263,111541,4,0,0,4400726.0,2021,1,1,6,3,...,1,0,0,0,0,0,"791154,679","5824616,757","13,28949592","52,49381321"
11264,110370,3,0,0,2400520.0,2021,1,20,1,3,...,1,0,0,0,0,1,"799988,2122","5828912,431","13,42309463","52,52753402"
11265,109998,7,0,0,7300619.0,2021,1,14,7,3,...,1,0,0,0,0,1,"796018,2589","5822739,196","13,35925879","52,47436651"


In [10]:
# changing column istsonstige to int (1 buggy cell insight)
def change_to_int(x):
    if isinstance(x, int):
        return x
    else: 
        if x == '0':
            x = int(0)
        elif x == '1':
            x = int(1)
        else:
            x = int(0)   

        return x 

In [11]:
accident_data['lor'] = accident_data['lor'].astype(float)
accident_data['lor'] = accident_data['lor'].astype('Int64')

accident_data['lor_ab_2021'] = accident_data['lor_ab_2021'].astype(float)
accident_data['lor_ab_2021'] = accident_data['lor_ab_2021'].astype('Int64')

accident_data['lor_ab_2021'] = accident_data['lor_ab_2021'].astype(float)
accident_data['lor_ab_2021'] = accident_data['lor_ab_2021'].astype('Int64')


accident_data['istsonstige'] = accident_data['istsonstige'].apply(lambda x: change_to_int(x))
accident_data['ustrzustand'] = accident_data['ustrzustand'].apply(lambda x: change_to_int(x))


accident_data['linrefx'] = accident_data['linrefx'].str.replace(",",'.').astype(float)
accident_data['linrefy'] = accident_data['linrefy'].str.replace(",",'.').astype(float)
accident_data['xgcswgs84'] = accident_data['xgcswgs84'].str.replace(",",'.').astype(float)
accident_data['ygcswgs84'] = accident_data['ygcswgs84'].str.replace(",",'.').astype(float)

In [12]:
accident_data.to_csv('../data/output/raw_accident_dataset_2018-2021.csv', index = False)

display(accident_data.head())
print(accident_data.dtypes)
accident_data.columns




,objectid,bez,lor,strasse,lor_ab_2021,ujahr,umonat,ustunde,uwochentag,ukategorie,...,istpkw,istfuss,istkrad,istgkfz,istsonstige,ustrzustand,linrefx,linrefy,xgcswgs84,ygcswgs84
0,112695,2,2050602,Samariterviertel,2500729,2018,1,15,4,3,...,1,1,0,0,0,1,803604.1562,5827580.738,13.475018,52.513597
1,112705,12,12304314,Ziekowstraße/Freie Scholle,12500824,2018,1,11,2,3,...,1,0,0,0,0,0,790639.5854,5835009.589,13.291022,52.587259
2,112726,2,2040501,Barnimkiez,2400520,2018,1,9,3,3,...,1,1,0,0,0,0,799827.9618,5828733.523,13.420578,52.526019
3,112737,7,7020202,Volkspark (Rudolf-Wilde-Park),7200308,2018,1,17,2,3,...,1,1,0,0,0,0,795223.5090,5823525.402,13.348288,52.481844
4,112747,3,3020209,Niederschönhausen,3200206,2018,1,15,4,3,...,0,1,0,0,0,1,798261.3849,5835047.260,13.403228,52.583472


objectid         int64
bez              int64
lor              Int64
strasse         object
lor_ab_2021      Int64
ujahr            int64
umonat           int64
ustunde          int64
uwochentag       int64
ukategorie       int64
uart             int64
utyp1            int64
ulichtverh       int64
istrad           int64
istpkw           int64
istfuss          int64
istkrad          int64
istgkfz          int64
istsonstige      int64
ustrzustand      int64
linrefx        float64
linrefy        float64
xgcswgs84      float64
ygcswgs84      float64
dtype: object


Index(['objectid', 'bez', 'lor', 'strasse', 'lor_ab_2021', 'ujahr', 'umonat',
       'ustunde', 'uwochentag', 'ukategorie', 'uart', 'utyp1', 'ulichtverh',
       'istrad', 'istpkw', 'istfuss', 'istkrad', 'istgkfz', 'istsonstige',
       'ustrzustand', 'linrefx', 'linrefy', 'xgcswgs84', 'ygcswgs84'],
      dtype='object')

In [13]:

accident_data = accident_data.drop(columns=['bez', 'lor'])
print(accident_data.columns)

Index(['objectid', 'strasse', 'lor_ab_2021', 'ujahr', 'umonat', 'ustunde',
       'uwochentag', 'ukategorie', 'uart', 'utyp1', 'ulichtverh', 'istrad',
       'istpkw', 'istfuss', 'istkrad', 'istgkfz', 'istsonstige', 'ustrzustand',
       'linrefx', 'linrefy', 'xgcswgs84', 'ygcswgs84'],
      dtype='object')


In [14]:


accident_data.columns = ['object_id', 'street_default', 'lor', 'year', 'month',
       'hour', 'weekday', 'ac_category', 'ac_type', 'ac_type2', 'ac_light',
       'is_bicycle', 'is_car', 'is_pedestrian', 'is_motorcycle', 'is_truck', 'is_other',
       'street_condition', 'linrefx', 'linrefy', 'xgcswgs84', 'ygcswgs84']

print(accident_data.columns)
display(accident_data)

filter = accident_data[accident_data['lor'].isnull()]
filter

key = accident_data['object_id'].astype(str) + '-' + accident_data['year'].astype(str) + '-' + accident_data['lor'].astype(str)

accident_data.insert(0, 'key', key)

display(accident_data)
accident_data.dtypes



Index(['object_id', 'street_default', 'lor', 'year', 'month', 'hour',
       'weekday', 'ac_category', 'ac_type', 'ac_type2', 'ac_light',
       'is_bicycle', 'is_car', 'is_pedestrian', 'is_motorcycle', 'is_truck',
       'is_other', 'street_condition', 'linrefx', 'linrefy', 'xgcswgs84',
       'ygcswgs84'],
      dtype='object')


,object_id,street_default,lor,year,month,hour,weekday,ac_category,ac_type,ac_type2,...,is_car,is_pedestrian,is_motorcycle,is_truck,is_other,street_condition,linrefx,linrefy,xgcswgs84,ygcswgs84
0,112695,Samariterviertel,2500729,2018,1,15,4,3,6,4,...,1,1,0,0,0,1,803604.1562,5827580.738,13.475018,52.513597
1,112705,Ziekowstraße/Freie Scholle,12500824,2018,1,11,2,3,2,6,...,1,0,0,0,0,0,790639.5854,5835009.589,13.291022,52.587259
2,112726,Barnimkiez,2400520,2018,1,9,3,3,6,4,...,1,1,0,0,0,0,799827.9618,5828733.523,13.420578,52.526019
3,112737,Volkspark (Rudolf-Wilde-Park),7200308,2018,1,17,2,3,6,7,...,1,1,0,0,0,0,795223.5090,5823525.402,13.348288,52.481844
4,112747,Niederschönhausen,3200206,2018,1,15,4,3,6,7,...,0,1,0,0,0,1,798261.3849,5835047.260,13.403228,52.583472
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11262,112174,0,4300414,2021,3,16,4,3,6,2,...,1,1,0,0,0,1,791505.4105,5826555.664,13.296345,52.511008
11263,111541,0,4400726,2021,1,1,6,3,9,1,...,1,0,0,0,0,0,791154.6790,5824616.757,13.289496,52.493813
11264,110370,0,2400520,2021,1,20,1,3,3,6,...,1,0,0,0,0,1,799988.2122,5828912.431,13.423095,52.527534
11265,109998,0,7300619,2021,1,14,7,3,2,6,...,1,0,0,0,0,1,796018.2589,5822739.196,13.359259,52.474367


,key,object_id,street_default,lor,year,month,hour,weekday,ac_category,ac_type,...,is_car,is_pedestrian,is_motorcycle,is_truck,is_other,street_condition,linrefx,linrefy,xgcswgs84,ygcswgs84
0,112695-2018-2500729,112695,Samariterviertel,2500729,2018,1,15,4,3,6,...,1,1,0,0,0,1,803604.1562,5827580.738,13.475018,52.513597
1,112705-2018-12500824,112705,Ziekowstraße/Freie Scholle,12500824,2018,1,11,2,3,2,...,1,0,0,0,0,0,790639.5854,5835009.589,13.291022,52.587259
2,112726-2018-2400520,112726,Barnimkiez,2400520,2018,1,9,3,3,6,...,1,1,0,0,0,0,799827.9618,5828733.523,13.420578,52.526019
3,112737-2018-7200308,112737,Volkspark (Rudolf-Wilde-Park),7200308,2018,1,17,2,3,6,...,1,1,0,0,0,0,795223.5090,5823525.402,13.348288,52.481844
4,112747-2018-3200206,112747,Niederschönhausen,3200206,2018,1,15,4,3,6,...,0,1,0,0,0,1,798261.3849,5835047.260,13.403228,52.583472
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11262,112174-2021-4300414,112174,0,4300414,2021,3,16,4,3,6,...,1,1,0,0,0,1,791505.4105,5826555.664,13.296345,52.511008
11263,111541-2021-4400726,111541,0,4400726,2021,1,1,6,3,9,...,1,0,0,0,0,0,791154.6790,5824616.757,13.289496,52.493813
11264,110370-2021-2400520,110370,0,2400520,2021,1,20,1,3,3,...,1,0,0,0,0,1,799988.2122,5828912.431,13.423095,52.527534
11265,109998-2021-7300619,109998,0,7300619,2021,1,14,7,3,2,...,1,0,0,0,0,1,796018.2589,5822739.196,13.359259,52.474367


In [29]:
from pyproj import Proj, Transformer
from tqdm import tqdm
# https://de.wikipedia.org/wiki/European_Petroleum_Survey_Group_Geodesy
# https://pyproj4.github.io/pyproj/stable/api/transformer.html
# https://pyproj4.github.io/pyproj/stable/api/proj.html
# https://gis.stackexchange.com/questions/326687/getting-epsg-code-in-pyproj-with-version-2-1-3
# https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
# https://pyproj4.github.io/pyproj/stable/gotchas.html#upgrading-to-pyproj-2-from-pyproj-1
# https://stackoverflow.com/questions/18603270/progress-indicator-during-pandas-operation.setdefault()
tqdm.pandas()

# ETRS89/UTM Zone 32N
utm = Proj('epsg:25832')

# WGS84
wgs = Proj('epsg:4326')
transformer = Transformer.from_crs('epsg:25832', 'epsg:4326')

lat, lon = transformer.transform(accident_data['linrefx'].tolist(), accident_data['linrefy'].tolist())

print(lat)
print(lon)

accident_data['latitude'] = lat
accident_data['longitude'] = lon

accident_data

[52.513596811254864, 52.587259063869666, 52.526018548945494, 52.481844469735506, 52.58347153761393, 52.551094599845555, 52.485715410637916, 52.633822610837896, 52.51893299339252, 52.573889046581364, 52.52360286780617, 52.470102585587945, 52.53525525877755, 52.514173128437854, 52.5108484577581, 52.54560284697962, 52.45899296727292, 52.516092072280074, 52.47011861258437, 52.50495533349049, 52.515912804318845, 52.59704720066088, 52.436302056714105, 52.54097318634199, 52.5536754720585, 52.45312997407885, 52.54813473463712, 52.52373969701958, 52.541111075721545, 52.51893644914078, 52.51878227303013, 52.61708256616523, 52.53276686035064, 52.5234457148849, 52.51108676283864, 52.536115927307826, 52.51883086211474, 52.54686536454658, 52.51764210503167, 52.48690005121837, 52.568172420336246, 52.469989789045364, 52.59762123978116, 52.55858736428614, 52.53209046076696, 52.497092325330634, 52.64245276827448, 52.43899810717756, 52.479704520952744, 52.42985298855452, 52.53584245774823, 52.52346812936

,key,object_id,street_default,lor,year,month,hour,weekday,ac_category,ac_type,...,is_motorcycle,is_truck,is_other,street_condition,linrefx,linrefy,xgcswgs84,ygcswgs84,latitude,longitude
0,112695-2018-2500729,112695,Samariterviertel,2500729,2018,1,15,4,3,6,...,0,0,0,1,803604.1562,5827580.738,13.475018,52.513597,52.513597,13.475018
1,112705-2018-12500824,112705,Ziekowstraße/Freie Scholle,12500824,2018,1,11,2,3,2,...,0,0,0,0,790639.5854,5835009.589,13.291022,52.587259,52.587259,13.291022
2,112726-2018-2400520,112726,Barnimkiez,2400520,2018,1,9,3,3,6,...,0,0,0,0,799827.9618,5828733.523,13.420578,52.526019,52.526019,13.420578
3,112737-2018-7200308,112737,Volkspark (Rudolf-Wilde-Park),7200308,2018,1,17,2,3,6,...,0,0,0,0,795223.5090,5823525.402,13.348288,52.481844,52.481844,13.348288
4,112747-2018-3200206,112747,Niederschönhausen,3200206,2018,1,15,4,3,6,...,0,0,0,1,798261.3849,5835047.260,13.403228,52.583472,52.583472,13.403228
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11262,112174-2021-4300414,112174,0,4300414,2021,3,16,4,3,6,...,0,0,0,1,791505.4105,5826555.664,13.296345,52.511008,52.511008,13.296345
11263,111541-2021-4400726,111541,0,4400726,2021,1,1,6,3,9,...,0,0,0,0,791154.6790,5824616.757,13.289496,52.493813,52.493813,13.289496
11264,110370-2021-2400520,110370,0,2400520,2021,1,20,1,3,3,...,0,0,0,1,799988.2122,5828912.431,13.423095,52.527534,52.527534,13.423095
11265,109998-2021-7300619,109998,0,7300619,2021,1,14,7,3,2,...,0,0,0,1,796018.2589,5822739.196,13.359259,52.474367,52.474367,13.359259


In [31]:
accident_data['latitude']

accident_data.columns

list = ['is_bicycle', 'is_car', 'is_pedestrian', 'is_motorcycle', 'is_truck',
       'is_other']

for l in list:
    accident_data[l] = accident_data[l].astype(bool)
    
    

In [34]:
accident_data
accident_data.columns

list = ['weekday', 'ac_category', 'ac_type', 'ac_type2', 'ac_light', 'street_condition']

for l in list:
    accident_data[l] = accident_data[l].astype('object')
    
    

In [35]:
accident_data.to_csv('../data/output/accident_dataset_2018-2021.csv', index = False)
accident_data.to_pickle('../data/output/accident_dataset_2018-2021.pkl')